In [1]:
# Cell 1: Import necessary libraries
import os
import sys
import torch
from transformers import AutoTokenizer, AutoConfig, DebertaForSequenceClassification

In [2]:
project_root = "/Users/tayebekavousi/Desktop/github_sa"
# Save the original directory to go back to it later if needed
original_dir = os.getcwd()
# Change to the project root directory
os.chdir(project_root)
# Ensure the project root is in the Python path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
sys.path.insert(0, '')  # Add current directory (empty string) to path


print("Environment setup complete!")

Environment setup complete!


In [5]:
from transformers import DebertaForSequenceClassification, DebertaTokenizerFast
import torch

# Step 1: Init model with same architecture
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=3)

# Step 2: Load saved checkpoint (just the state_dict, not full model)
checkpoint_path = "/Users/tayebekavousi/Desktop/github_sa/saved_models/deberta_3class/deberta_5epochs/model_best_f1_0.8954.pt"
checkpoint = torch.load(checkpoint_path, map_location="cpu")
model.load_state_dict(checkpoint["model_state_dict"], strict=False)  # 👈 add strict=False to avoid minor key mismatch

# Step 3: Ready for inference
model.eval()

# Optional: load tokenizer
tokenizer = DebertaTokenizerFast.from_pretrained("microsoft/deberta-base")

print("✅ DeBERTa model + tokenizer loaded from .pt checkpoint.")


Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'classifi

✅ DeBERTa model + tokenizer loaded from .pt checkpoint.


In [9]:
import torch
import torch.nn.functional as F

# Define your class names in the correct order (based on training config)
class_names = ["Negative", "Neutral", "Positive"]  # 👈 update if your order differs

# Sample input
sample_text = "well , as long as you ok with it lgtm!"

# Tokenize
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True)

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = F.softmax(logits, dim=1)

# Extract prediction
predicted_class_idx = torch.argmax(probs, dim=1).item()
predicted_class_name = class_names[predicted_class_idx]
confidence = probs[0][predicted_class_idx].item() * 100

# Output
print(f"🧠 Text: \"{sample_text}\"")
print(f"✅ Predicted Class: {predicted_class_name} ({predicted_class_idx})")
print(f"📊 Confidence: {confidence:.2f}%")


🧠 Text: "well , as long as you ok with it lgtm!"
✅ Predicted Class: Positive (2)
📊 Confidence: 87.18%
